In [5]:
# Celda 1: Importaciones, Carga y Preparación Completa de Datos

import os
import re
import pandas as pd
import faiss
import psutil
from dotenv import load_dotenv
from tqdm.auto import tqdm
import gradio as gr
from IPython.display import display
import unicodedata

# LlamaIndex Core y componentes
from llama_index.core import (
    VectorStoreIndex, StorageContext, Settings, Document,
    load_index_from_storage
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.prompts import PromptTemplate
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.openrouter import OpenRouter
from llama_index.llms.google_genai import GoogleGenAI

# --- 1. Configurar Entorno ---
num_cores = psutil.cpu_count(logical=True)
os.environ['OMP_NUM_THREADS'] = str(num_cores)
os.environ['MKL_NUM_THREADS'] = str(num_cores)
print(f"Librerías importadas. ¡Entorno listo con {num_cores} núcleos disponibles!")

# --- 2. Funciones de Preparación de Datos ---
METADATA_FILE = os.path.join("data", "metadatos_chatbot_final.csv")

def normalize_text(text):
    """Convierte texto a minúsculas y elimina acentos."""
    if not isinstance(text, str):
        return ""
    return ''.join(
        c for c in unicodedata.normalize('NFD', text)
        if unicodedata.category(c) != 'Mn'
    ).lower()

def load_and_prepare_metadata(filepath):
    """
    Carga los metadatos desde un CSV y realiza toda la preparación necesaria:
    - Rellena valores nulos.
    - Crea columnas normalizadas para la búsqueda.
    - Crea una columna 'Autoría' unificada y limpia a partir de las normalizadas.
    """
    try:
        df = pd.read_csv(filepath)
        df.fillna("No especificado", inplace=True)
        print("Metadatos cargados y procesados.")

        # --- Creación de columnas normalizadas para búsqueda eficiente ---
        print("Creando columnas normalizadas para la búsqueda de autores...")
        df['principal_norm'] = df['Investigador u organismo principal'].apply(normalize_text)
        df['equipo_norm'] = df['Equipo de investigación'].apply(normalize_text)
        print("Columnas normalizadas creadas exitosamente.")

        # --- Creación de la columna de autoría combinada y normalizada ---
        print("Creando columna de 'Autoría_norm' combinada...")
        
        def limpiar_y_unir_autores_normalizados(row):
            # --- CAMBIO CLAVE: Usamos las columnas ya normalizadas ---
            autores_raw_norm = f"{row['principal_norm']};{row['equipo_norm']}"
            partes = [a.strip() for a in re.split(r'[;,]', autores_raw_norm)]
            # Eliminar vacíos y los valores por defecto ya normalizados
            partes_limpias = [p for p in partes if p and p != "no especificado" and p != "sin informacion"]
            
            # Quitar duplicados (ya no es necesario, pero es una buena práctica)
            autores_unicos = sorted(list(set(partes_limpias)))
            
            return ", ".join(autores_unicos) if autores_unicos else "sin informacion"
        
        # Creamos una columna de autoría ya normalizada para las búsquedas
        df['Autoría_norm'] = df.apply(limpiar_y_unir_autores_normalizados, axis=1)

        # También creamos una columna 'Autoría' legible para mostrar al usuario
        df['Autoría'] = df.apply(
            lambda row: ", ".join(filter(None, [
                row['Investigador u organismo principal'] if row['Investigador u organismo principal'] != 'No especificado' else None,
                row['Equipo de investigación'] if row['Equipo de investigación'] != 'No especificado' else None
            ])) or "Sin información",
            axis=1
        )
        
        print("Columnas de autoría creadas exitosamente.")
        
        display(df[['Investigador u organismo principal', 'Equipo de investigación', 'Autoría', 'Autoría_norm']].head())
        return df

    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo de metadatos en la ruta: {filepath}")
        return None
    except Exception as e:
        print(f"ERROR al cargar metadatos: {e}.")
        return None

# --- Ejecución ---
# Cargar y preparar el DataFrame en un solo paso.
df_metadatos = load_and_prepare_metadata(METADATA_FILE)

Librerías importadas. ¡Entorno listo con 6 núcleos disponibles!
Metadatos cargados y procesados.
Creando columnas normalizadas para la búsqueda de autores...
Columnas normalizadas creadas exitosamente.
Creando columna de 'Autoría_norm' combinada...
Columnas de autoría creadas exitosamente.


,Investigador u organismo principal,Equipo de investigación,Autoría,Autoría_norm
0,"Bravo, David","Ruiz-Tagle, Jaime; Sanhueza, Ricardo","Bravo, David, Ruiz-Tagle, Jaime; Sanhueza, Ric...","bravo, david, jaime, ricardo, ruiz-tagle, sanh..."
1,"Raczynski, Dagmar","Pavez, M. Angélica","Raczynski, Dagmar, Pavez, M. Angélica","dagmar, m. angelica, pavez, raczynski"
2,"Marshall, Guillermo","Correa, Lorena","Marshall, Guillermo, Correa, Lorena","correa, guillermo, lorena, marshall"
3,"Marshall, Guillermo","Correa, Lorena","Marshall, Guillermo, Correa, Lorena","correa, guillermo, lorena, marshall"
4,"Fernández, Carolina","Jashes, Jessana","Fernández, Carolina, Jashes, Jessana","carolina, fernandez, jashes, jessana"


In [2]:
# Celda 2: CONFIGURACIÓN OPTIMIZADA DE MODELOS DE IA

# Cargar la clave de API desde el archivo .env
load_dotenv()
if "OPENROUTER_API_KEY" in os.environ:
    print("Clave de API de OpenRouter cargada.")
else:
    print("¡ADVERTENCIA! No se encontró la clave de API de OpenRouter.")

if "HF_TOKEN" in os.environ:
    print("Token de Hugging Face (HF_TOKEN) encontrado en el entorno.")
else:
    print("¡ADVERTENCIA! No se encontró el token de Hugging Face.")

# --- Configuración global de LlamaIndex optimizada para CPU ---
optimal_batch_size = max(64, num_cores * 8)
print(f"Configurando el modelo de embeddings con batch size óptimo de {optimal_batch_size} para {num_cores} núcleos.")

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-small-en-v1.5",
    embed_batch_size=optimal_batch_size,
    cache_folder="./model_cache",
    normalize=True,
    trust_remote_code=True
)

# Selecciona el proveedor de LLM aquí: "openrouter" o "gemini"
LLM_PROVIDER = "gemini"

if LLM_PROVIDER == "gemini":
    Settings.llm = GoogleGenAI(
        api_key=os.getenv("GOOGLE_API_KEY"),
        model="gemini-1.5-flash-latest",
        temperature=0.1,
    )
    print("Usando Gemini como modelo LLM.")
else:
    Settings.llm = OpenRouter(
        model="google/gemma-3n-e4b-it:free",
        #model="google/gemma-3n-e2b-it:free",
        #model="mistralai/mistral-small-3.2-24b-instruct:free",
        #model="deepseek/deepseek-chat-v3-0324:free",
        #model="deepseek/deepseek-r1-0528:free",
        #model="deepseek/deepseek-chat:free",
        #model="google/gemini-2.0-flash-exp:free",
        #model="mistralai/mistral-nemo:free",
        #model="qwen/qwq-32b:free",
        #model="microsoft/mai-ds-r1:free",
        #model="meta-llama/llama-4-maverick:free",
        temperature=0.1, 
    )
    print("Usando OpenRouter como modelo LLM.")

# Función para monitorear uso de memoria
def print_memory_usage():
    process = psutil.Process(os.getpid())
    memory_info = process.memory_info()
    print(f"Uso de memoria actual: {memory_info.rss / (1024 * 1024):.2f} MB")

print("\nConfiguración de modelos completada.")
print_memory_usage()

Clave de API de OpenRouter cargada.
Token de Hugging Face (HF_TOKEN) encontrado en el entorno.
Configurando el modelo de embeddings con batch size óptimo de 64 para 6 núcleos.
Usando Gemini como modelo LLM.

Configuración de modelos completada.
Uso de memoria actual: 553.43 MB


In [3]:
# Celda 3: Creación o Carga PERSISTENTE del Índice Vectorial (Optimizada para Hugging Face Spaces)

# 1. Definimos las rutas donde se guardarán los componentes del índice
PERSIST_DIR = "./storage_index"
FAISS_INDEX_PATH = os.path.join(PERSIST_DIR, "faiss_index.bin")
DOCSTORE_PATH = os.path.join(PERSIST_DIR, "docstore.json")
INDEX_STORE_PATH = os.path.join(PERSIST_DIR, "index_store.json")

# Función para eliminar una carpeta y su contenido de forma segura
def remove_directory_safely(directory_path):
    import shutil
    try:
        if os.path.exists(directory_path):
            print(f"Eliminando directorio '{directory_path}'...")
            shutil.rmtree(directory_path)
            print(f"Directorio '{directory_path}' eliminado exitosamente.")
        return True
    except Exception as e:
        print(f"Error al eliminar directorio: {e}")
        return False

try:
    # Verificamos si existe el índice FAISS serializado
    if os.path.exists(FAISS_INDEX_PATH) and os.path.exists(DOCSTORE_PATH) and os.path.exists(INDEX_STORE_PATH):
        print(f"Cargando componentes del índice desde '{PERSIST_DIR}'...")
        try:
            # 1. Cargamos el índice FAISS con su método nativo
            faiss_index = faiss.read_index(FAISS_INDEX_PATH)
            print("Índice FAISS cargado correctamente.")
            
            # 2. Creamos el vector store y el storage context
            vector_store = FaissVectorStore(faiss_index=faiss_index)
            storage_context = StorageContext.from_defaults(
                vector_store=vector_store,
                persist_dir=PERSIST_DIR
            )
            
            # 3. Cargamos el índice completo
            metadata_index = load_index_from_storage(storage_context)
            print("¡Índice completo cargado exitosamente!")
            
        except Exception as e:
            print(f"Error al cargar algún componente del índice: {e}")
            if remove_directory_safely(PERSIST_DIR):
                print("Recreando el índice desde cero...")
            else:
                raise Exception("No se pudo eliminar el índice corrupto. Elimínalo manualmente.")
    
    # Si llegamos aquí, es porque necesitamos crear el índice (la carpeta no existe o se eliminó)
    if not os.path.exists(PERSIST_DIR) or not os.path.exists(FAISS_INDEX_PATH):
        print("Creando un nuevo índice...")
        
        if df_metadatos is None:
            print("No se puede crear el índice porque el DataFrame de metadatos no se cargó. Revisa la Celda 1.")
            metadata_index = None
        else:
            # --- Lógica de creación (la misma que tenías antes) ---
            print("="*50)
            print("INICIANDO PROCESO DE CREACIÓN DE ÍNDICE (LENTO LA PRIMERA VEZ)...\nEste proceso solo se ejecutará una vez.")
            print("="*50)

            Settings.node_parser = SentenceSplitter(chunk_size=2048)

            # La función optimize_text_for_embedding sigue siendo útil para el contenido de los nodos
            def optimize_text_for_embedding(text, max_length=1500):
                if not isinstance(text, str): return ""
                text = text[:max_length]
                text = re.sub(r'\s+', ' ', text)
                return text.strip()
                
            print("Convirtiendo metadatos en documentos de LlamaIndex...")
            metadata_documents = []
            columnas_a_incluir = [
                'Año de publicación', 'Código sugerido', 'Nombre del archivo', 'Título del estudio', 
                'Categoría', 'Tipo de documento', 'Subcategoría', 'Idioma', 'Número de páginas', 
                'Incorpora perspectiva de género', 'Año de término', 'Lugar de término', 
                'Palabras clave', 'Objetivo', 'Metodología', 'Resumen', 'Documento público', 
                'Publicación destacada', 'Publicado', 'Editorial', 'Entidad solicitante',
                'Entidad a cargo del estudio', 'Investigador u organismo principal', 'Equipo de investigación', 
                'Número de documento administrativo', 'Tipo de financiamiento', 'Costo del estudio', 
                'Base de datos enviada', 'Base pública', 'Url'
            ]
            columnas_prioritarias = ['Título del estudio', 'Resumen', 'Palabras clave', 'Objetivo', 
                                    'Metodología', 'Investigador u organismo principal']

            for index, row in tqdm(df_metadatos.iterrows(), total=df_metadatos.shape[0], desc="Procesando metadatos"):
                partes_texto = []
                for col in columnas_prioritarias:
                    if col in row and pd.notna(row[col]) and row[col] != "No especificado":
                        texto_optimizado = optimize_text_for_embedding(str(row[col]), 300)
                        if texto_optimizado:
                            partes_texto.append(f"{col}: {texto_optimizado}"); partes_texto.append(f"{col}: {texto_optimizado}")
                
                for col in columnas_a_incluir:
                    if col not in columnas_prioritarias:
                        if col in row and pd.notna(row[col]) and row[col] != "No especificado":
                            texto_optimizado = optimize_text_for_embedding(str(row[col]), 200)
                            if texto_optimizado:
                                partes_texto.append(f"{col}: {texto_optimizado}")
                
                contenido_buscable = ". ".join(partes_texto)
                doc = Document(
                    text=contenido_buscable,
                    metadata={col: optimize_text_for_embedding(str(row.get(col, '')), 500) for col in columnas_a_incluir}
                )
                metadata_documents.append(doc)
            
            print("Creando índice vectorial con FAISS...")
            d = len(Settings.embed_model.get_text_embedding("test"))
            faiss_index = faiss.IndexFlatL2(d)
            vector_store = FaissVectorStore(faiss_index=faiss_index)
            storage_context = StorageContext.from_defaults(vector_store=vector_store)
            
            metadata_index = VectorStoreIndex.from_documents(
                metadata_documents,
                storage_context=storage_context,
                show_progress=True 
            )
            
            # --- ¡PASOS CRUCIALES DE GUARDADO! ---
            # 1. Creamos el directorio si no existe
            os.makedirs(PERSIST_DIR, exist_ok=True)
            
            # 2. Guardamos el índice FAISS usando su método nativo
            print(f"Guardando el índice FAISS en '{FAISS_INDEX_PATH}'...")
            faiss.write_index(faiss_index, FAISS_INDEX_PATH)
            
            # 3. Guardamos el resto del storage context (docstore, index_store)
            print(f"Guardando los componentes de LlamaIndex en '{PERSIST_DIR}'...")
            metadata_index.storage_context.persist(persist_dir=PERSIST_DIR)
            
            print("="*50)
            print("¡ÍNDICE CREADO Y GUARDADO EN FORMATO COMPATIBLE!")
            print("="*50)

except Exception as e:
    print(f"Error no manejado: {e}")
    print("Por favor, borra manualmente la carpeta 'storage_index' e intenta nuevamente.")
    metadata_index = None

Cargando componentes del índice desde './storage_index'...
Índice FAISS cargado correctamente.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage_index\docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from ./storage_index\index_store.json.
¡Índice completo cargado exitosamente!


In [4]:
# Celda 4: Lógica del Chatbot e Interfaz Gradio (Caja de Texto Limpia)

if 'metadata_index' not in locals() or metadata_index is None or 'df_metadatos' not in locals() or df_metadatos is None:
    print("ERROR: El índice o el DataFrame no han sido creados. Por favor, ejecuta las celdas anteriores correctamente.")
else:
    # --- FUNCIONES DE APOYO (Sin cambios) ---
    def normalize_text(text):
        if not isinstance(text, str): return ""
        return ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn').lower()

    def get_initial_chat_state():
        return {"user_name": None, "current_mode": "waiting_for_name", "temp_author": None,
                "last_query_term": None, "last_results": [], "current_page": 0, "last_search_type": None}
    
    def search_by_topic(message, author_filter=None):
        print(f"Búsqueda semántica por tema: '{message}' con filtro de autor: {author_filter}"); extraction_template = PromptTemplate("Reescribe la consulta del usuario en una lista de 3 a 5 términos de búsqueda concisos, separados por comas. Responde SÓLO con los términos.\nConsulta: '{query_str}'\nTérminos:"); response = Settings.llm.complete(extraction_template.format(query_str=message)); llm_term = str(response).strip().lower().replace('*', '').replace('-', ''); print(f"Término de búsqueda final: '{llm_term}'"); retriever = metadata_index.as_retriever(similarity_top_k=50); retrieved_nodes = retriever.retrieve(llm_term); results = []
        if retrieved_nodes:
            normalized_author_filter_parts = normalize_text(author_filter).split() if author_filter else []
            for node_with_score in retrieved_nodes:
                if normalized_author_filter_parts:
                    full_text_norm = normalize_text(node_with_score.node.text)
                    if not all(part in full_text_norm for part in normalized_author_filter_parts): continue
                meta = node_with_score.node.metadata
                results.append({'Título del estudio': meta.get('Título del estudio', 'N/A'), 'Autor(es)': f"{meta.get('Investigador u organismo principal', 'N/A')}, {meta.get('Equipo de investigación', 'N/A')}", 'Año de publicación': meta.get('Año de publicación', 'N/A'), 'Url': meta.get('Url', 'No disponible'), '__is_semantic': True, 'score': node_with_score.score})
        results.sort(key=lambda x: x['score'], reverse=True); relevant_results = [res for res in results if res["score"] >= 0.40]
        return {"results": relevant_results, "query_term": llm_term}
    
    def search_by_author(authors_query):
        print(f"Búsqueda por autor/es: '{authors_query}'."); author_groups = [group.strip() for group in authors_query.split(',') if group.strip()]
        def df_to_results_list(df):
            df_copy = df.copy(); df_copy.loc[:, 'Autor(es)'] = df_copy.apply(lambda row: f"{row.get('Investigador u organismo principal', 'N/A')}, {row.get('Equipo de investigación', 'N/A')}", axis=1); return df_copy.to_dict('records')
        and_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
        for group in author_groups:
            normalized_group_parts = normalize_text(group).split(); group_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
            for part in normalized_group_parts: group_mask &= (df_metadatos['principal_norm'].str.contains(part, na=False) | df_metadatos['equipo_norm'].str.contains(part, na=False))
            and_mask &= group_mask
        and_hits_df = df_metadatos[and_mask].copy()
        if not and_hits_df.empty:
            and_hits_df.loc[:, 'Año de publicación num'] = pd.to_numeric(and_hits_df['Año de publicación'], errors='coerce').fillna(0); sorted_df = and_hits_df.sort_values(by='Año de publicación num', ascending=False); results_list = df_to_results_list(sorted_df); header = f"He encontrado {len(results_list)} documento(s) que incluyen a '{authors_query}'."; return {"results": results_list, "header": header}
        if len(author_groups) > 1:
            response_parts = [f"No he encontrado documentos que incluyan a todos estos autores juntos. A continuación, los resultados para cada uno por separado:\n"]; any_found = False
            for group in author_groups:
                normalized_group_parts = normalize_text(group).split(); group_mask = pd.Series([True] * len(df_metadatos), index=df_metadatos.index)
                for part in normalized_group_parts: group_mask &= (df_metadatos['principal_norm'].str.contains(part, na=False) | df_metadatos['equipo_norm'].str.contains(part, na=False))
                individual_hits_df = df_metadatos[group_mask].copy()
                if not individual_hits_df.empty:
                    any_found = True; total_individual = len(individual_hits_df); limit_text = f"(mostrando los 5 más recientes de {total_individual})" if total_individual > 5 else ""; response_parts.append(f"\n---\n\n### Documentos de '{group}' {limit_text}:\n"); individual_hits_df.loc[:, 'Año de publicación num'] = pd.to_numeric(individual_hits_df['Año de publicación'], errors='coerce').fillna(0); sorted_individual_df = individual_hits_df.sort_values(by='Año de publicación num', ascending=False).head(5)
                    for i, (_, row) in enumerate(sorted_individual_df.iterrows(), 1): response_parts.append(f"{i}. **Título:** {row.get('Título del estudio', 'N/A')}\n   - **Autor(es):** {row.get('Investigador u organismo principal', 'N/A')}, {row.get('Equipo de investigación', 'N/A')}\n   - **Año:** {row.get('Año de publicación', 'N/A')}\n   - **URL:** {row.get('Url', 'No disponible')}\n")
                else: response_parts.append(f"\n---\n\n### Documentos de '{group}':\n\nNo se encontraron documentos.")
            if any_found: return {"results": [], "header": "\n".join(response_parts), "is_special_format": True}
        return {"results": [], "header": f"No he encontrado documentos de '{authors_query}'. Revisa la ortografía."}
    
    def format_results_page(state):
        total_found = len(state["last_results"]);
        if total_found == 0: return state.get("custom_header") or "No se encontraron resultados."
        MAX_TO_SHOW = 5; start_index = (state["current_page"] - 1) * MAX_TO_SHOW; nodes_to_show = state["last_results"][start_index : start_index + MAX_TO_SHOW]
        if not nodes_to_show: return "No hay más documentos que mostrar. Ya estás en la última página."
        header_parts = [state.get("custom_header", f"Mostrando resultados para '{state['last_query_term']}'")]
        if total_found > MAX_TO_SHOW: header_parts.append(f"(página {state['current_page']} de {-(-total_found // MAX_TO_SHOW)}, mostrando {len(nodes_to_show)} de {total_found} total)")
        header = " ".join(filter(None, header_parts)) + ":\n\n"; lista_formateada = []
        for i, res in enumerate(nodes_to_show, start=start_index + 1):
            titulo = res.get('Título del estudio', 'N/A'); autores = res.get('Autor(es)', 'N/A'); ano = str(res.get('Año de publicación', 'N/A')); url = res.get('Url', 'No disponible')
            if res.get('__is_semantic'): score = res.get('score', 0); relevance_label = "Muy Alta" if score >= 0.75 else "Alta" if score >= 0.5 else "Media"; relevance_str = f"**(Relevancia: {relevance_label} [{score:.0%}])**"; lista_formateada.append(f"{i}. **Título:** {titulo} {relevance_str}\n   - **Autor(es):** {autores}\n   - **Año:** {ano}\n   - **URL:** {url}")
            else: lista_formateada.append(f"{i}. **Título:** {titulo}\n   - **Autor(es):** {autores}\n   - **Año:** {ano}\n   - **URL:** {url}")
        footer = f"\n\n--- \n*Para ver más, escribe **siguiente** o `página {state['current_page'] + 1}`.*" if (start_index + MAX_TO_SHOW) < total_found else ""
        return header + "\n\n".join(lista_formateada) + footer
    
    def extract_name_from_greeting(message):
        patterns = [r"(?:soy|me llamo|mi nombre es|ll[aá]mame|dime)\s+(.+)", r"^(?:hola,? soy|hola,? mi nombre es)\s+(.+)"];
        for pattern in patterns: match = re.search(pattern, message, re.IGNORECASE);
        if match: return match.group(1).strip().title()
        return message.strip().title()

    with gr.Blocks(theme="default") as demo:
        chat_state = gr.State(get_initial_chat_state())
        chatbot = gr.Chatbot(value=[{"role": "assistant", "content": "¡Hola! Soy CemIA, asistente de IA del Centro de Estudios del MINEDUC. ¿Cómo quieres que te llame?"}], type="messages", label="CEMIA", height=500)
        with gr.Row(visible=False) as button_row:
            btn_author = gr.Button("Búsqueda por autoría"); btn_topic = gr.Button("Búsqueda por temática"); btn_combo = gr.Button("Búsqueda por autoría y temática"); btn_bye = gr.Button("Adiós")
        user_input = gr.Textbox(placeholder="Escribe tu nombre para comenzar...", label="Tu respuesta", autofocus=True)

        def make_buttons_interactive(interactive=True):
            return gr.update(interactive=interactive), gr.update(interactive=interactive), gr.update(interactive=interactive), gr.update(interactive=interactive)

        def handle_user_input(message, history, current_state):
            history.append({"role": "user", "content": message}); mode = current_state["current_mode"]
            
            message_lower = message.strip().lower()
            if message_lower in ["siguiente", "más", "mas"] or re.search(r"p[aá]gina\s+(\d+)", message_lower):
                if current_state["last_results"]:
                    page_match = re.search(r"p[aá]gina\s+(\d+)", message_lower); current_state["current_page"] = int(page_match.group(1)) if page_match else current_state["current_page"] + 1
                    history.append({"role": "assistant", "content": format_results_page(current_state)})
                else: history.append({"role": "assistant", "content": "No hay resultados anteriores para paginar. Por favor, realiza una nueva búsqueda."})
                return history, current_state, gr.update(), gr.update(), gr.update(), gr.update(), gr.update(), gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción")

            button_row_update, user_input_update = gr.update(), gr.update(value="", interactive=False, placeholder="Elige una opción de los botones");
            btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update = make_buttons_interactive(False); response_text = ""

            if mode == "waiting_for_name":
                user_name = extract_name_from_greeting(message); current_state["user_name"], current_state["current_mode"] = user_name, "waiting_for_choice"
                response_text = f"¡Qué bueno tenerte aquí, {user_name}! Por favor, elige una opción."; button_row_update, (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = gr.update(visible=True), make_buttons_interactive(True)
            elif mode == "waiting_for_author":
                search_data = search_by_author(message)
                if search_data.get("is_special_format"): response_text = search_data["header"]; current_state["last_results"] = []
                else: current_state.update({"last_results": search_data["results"], "last_query_term": message, "current_page": 1, "custom_header": search_data["header"], "last_search_type": "author"}); response_text = format_results_page(current_state)
                current_state["current_mode"] = "waiting_for_choice"; (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True); user_input_update = gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción", interactive=True)
            elif mode == "waiting_for_topic":
                search_data = search_by_topic(message); current_state.update({"last_results": search_data["results"], "last_query_term": search_data["query_term"], "current_page": 1, "last_search_type": "topic", "current_mode": "waiting_for_choice"}); response_text = format_results_page(current_state)
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True); user_input_update = gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción", interactive=True)
            elif mode == "waiting_for_combo_author":
                current_state["temp_author"], current_state["current_mode"] = message.strip(), "waiting_for_combo_topic"; response_text = "Entendido. Ahora dime la temática que quieres buscar."; user_input_update = gr.update(value="", interactive=True, placeholder="Escribe la temática aquí...")
            elif mode == "waiting_for_combo_topic":
                author = current_state["temp_author"]; search_data = search_by_topic(message, author_filter=author); current_state.update({"last_results": search_data["results"], "last_query_term": search_data["query_term"], "current_page": 1, "custom_header": f"Resultados para '{search_data['query_term']}' filtrados por el autor '{author}'", "last_search_type": "topic", "current_mode": "waiting_for_choice", "temp_author": None}); response_text = format_results_page(current_state)
                (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True); user_input_update = gr.update(value="", placeholder="Escribe 'siguiente' o elige una opción", interactive=True)
            else: response_text = "Por favor, utiliza uno de los botones para continuar."; (btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update) = make_buttons_interactive(True)
            
            history.append({"role": "assistant", "content": response_text})
            return history, current_state, button_row_update, btn_author_update, btn_topic_update, btn_combo_update, btn_bye_update, user_input_update

        def handle_choice(history, current_state, choice):
            history.append({"role": "assistant", "content": f"Has elegido: **{choice}**"}); prompt_message, mode = "", current_state["current_mode"]
            if choice == "Búsqueda por autoría": mode, prompt_message = "waiting_for_author", "Por favor, escribe el nombre del autor/a o autores/as (separados por comas)."
            elif choice == "Búsqueda por temática": mode, prompt_message = "waiting_for_topic", "Por favor, escribe la temática que te interesa."
            elif choice == "Búsqueda por autoría y temática": mode, prompt_message = "waiting_for_combo_author", "Perfecto. Primero, escribe el nombre del autor."
            current_state["current_mode"] = mode; history.append({"role": "assistant", "content": prompt_message}); b1, b2, b3, b4 = make_buttons_interactive(False)
            
            # --- CAMBIO AQUÍ ---
            return history, current_state, b1, b2, b3, b4, gr.update(value="", interactive=True, placeholder="Escribe aquí...")

        def handle_bye(current_state):
            if current_state["user_name"]:
                gr.Info(f"Adiós {current_state['user_name']}, ¡recuerda que siempre estoy disponible para asistirte!")
                current_state["current_mode"] = "ended"
            
            b1, b2, b3, b4 = make_buttons_interactive(False)
            ui_input = gr.update(value="", interactive=False, placeholder="Conversación finalizada. Recarga la página para empezar de nuevo.")
            return current_state, b1, b2, b3, b4, ui_input

        user_input.submit(handle_user_input, [user_input, chatbot, chat_state], [chatbot, chat_state, button_row, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_author.click(lambda h, s: handle_choice(h, s, "Búsqueda por autoría"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_topic.click(lambda h, s: handle_choice(h, s, "Búsqueda por temática"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_combo.click(lambda h, s: handle_choice(h, s, "Búsqueda por autoría y temática"), [chatbot, chat_state], [chatbot, chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])
        btn_bye.click(fn=handle_bye, inputs=[chat_state], outputs=[chat_state, btn_author, btn_topic, btn_combo, btn_bye, user_input])

    print("\nLanzando la interfaz de CEMIA con botones...")
    demo.launch(inline=True, share=False)


Lanzando la interfaz de CEMIA con botones...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
